In [ ]:
!pip install scikit-multilearn
!pip install scikit-multilearn --upgrade

import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import sqlite3
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
import nltk
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime
from skmultilearn.adapt import mlknn
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split


from google.colab import drive
drive.mount('/content/drive')

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

Requirement already up-to-date: scikit-multilearn in /usr/local/lib/python3.7/dist-packages (0.2.0)
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
text_path='/content/drive/MyDrive/colab_SMAI/test1.txt'
df=pd.read_csv(text_path, names=['Id', 'Title','Body','Tag'],skiprows=1,nrows=1200000)
df['Tag'] = df['Tag'].fillna("")
df['Tag'] = df['Tag'].astype(str)

df['cleaned_tags']=df['Tag'].str.lower().replace('\n','')
df['cleaned_tags']=df['cleaned_tags'].apply(lambda x: str(x).encode('ascii', 'ignore').decode('ascii'))
df['cleaned_tags']=df['cleaned_tags'].str.replace('\d+', '')
df['cleaned_tags']=df['cleaned_tags'].str.replace(' +', ' ')

junk_chars = "[]{}.-"
for c in junk_chars:
  df['cleaned_tags']=df['cleaned_tags'].str.replace(c, '')

qus_list=[]
qus_with_code = 0
len_before_preprocessing = 0 
len_after_preprocessing = 0 
cnt = 0
for index,row in df.iterrows():
    title, body, tags = row["Title"], row["Body"], row["cleaned_tags"]
    if '<code>' in body:
        qus_with_code+=1
    len_before_preprocessing+=len(title) + len(body)
    body=re.sub('<code>(.*?)</code>', '', body, flags=re.MULTILINE|re.DOTALL)
    body = re.sub('<.*?>', ' ', str(body.encode('utf-8')))
    title=title.encode('utf-8')
    question=str(title)+" "+str(body)
    question=re.sub(r'[^A-Za-z]+',' ',question)
    words=word_tokenize(str(question.lower()))
    question=' '.join(str(stemmer.stem(j)) for j in words if j not in stop_words and (len(j)!=1 or j=='c'))
    qus_list.append(question)
    len_after_preprocessing += len(question)
    cnt = cnt + 1
    if (cnt%25000 == 0):
      print("Qstn body processed : ", cnt)

df["question"] = qus_list
avg_len_before_preprocessing=(len_before_preprocessing*1.0)/df.shape[0]
avg_len_after_preprocessing=(len_after_preprocessing*1.0)/df.shape[0]
print(" total dataset size: "+ str(len(df)))

Qstn body processed :  25000
Qstn body processed :  50000
Qstn body processed :  75000
Qstn body processed :  100000
Qstn body processed :  125000
Qstn body processed :  150000
Qstn body processed :  175000
Qstn body processed :  200000
Qstn body processed :  225000
Qstn body processed :  250000
Qstn body processed :  275000
Qstn body processed :  300000
Qstn body processed :  325000
Qstn body processed :  350000
Qstn body processed :  375000
Qstn body processed :  400000
Qstn body processed :  425000
Qstn body processed :  450000
Qstn body processed :  475000
Qstn body processed :  500000
Qstn body processed :  525000
Qstn body processed :  550000
Qstn body processed :  575000
Qstn body processed :  600000
Qstn body processed :  625000
Qstn body processed :  650000
Qstn body processed :  675000
Qstn body processed :  700000
Qstn body processed :  725000
Qstn body processed :  750000
Qstn body processed :  775000
Qstn body processed :  800000
Qstn body processed :  825000
Qstn body pro

In [ ]:
preprocessed_df = df[["question","cleaned_tags"]]
preprocessed_df = preprocessed_df[:30000]
print("Shape of preprocessed data :", preprocessed_df.shape)

Shape of preprocessed data : (30000, 2)


In [ ]:
def tags_to_consider(n):
    tag_i_sum = y_multilabel.sum(axis=0).tolist()[0]
    sorted_tags_i = sorted(range(len(tag_i_sum)), key=lambda i: tag_i_sum[i], reverse=True)
    yn_multilabel=y_multilabel[:,sorted_tags_i[:n]]
    return yn_multilabel

def questions_covered_fn(numb):
    yn_multilabel = tags_to_consider(numb)
    x= yn_multilabel.sum(axis=1)
    return (np.count_nonzero(x==0))

vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')
y_multilabel = vectorizer.fit_transform(preprocessed_df['cleaned_tags'])

questions_covered = []
total_tags=y_multilabel.shape[1]
total_qus=preprocessed_df.shape[0]
for i in range(100, total_tags, 100):
    questions_covered.append(np.round(((total_qus-questions_covered_fn(i))/total_qus)*100,3))

yx_multilabel = tags_to_consider(100)
X_train, X_test, y_train, y_test = train_test_split(preprocessed_df, yx_multilabel, test_size = 0.1,random_state = 42)
vectorizer = TfidfVectorizer(min_df=0.00009, max_features=20000, tokenizer = lambda x: x.split(), ngram_range=(1,3))
X_train_multilabel = vectorizer.fit_transform(X_train['question'])
X_test_multilabel = vectorizer.transform(X_test['question'])

In [ ]:

cparam = [.65, .7, 1]
for cval in cparam:
  print(" ------------- for : "+ str(cval)+ "---------------")
  clf_lsvc = OneVsRestClassifier(LinearSVC(C=cval))
  clf_lsvc.fit(X_train_multilabel, y_train)
  y_pred = clf_lsvc.predict(X_test_multilabel)
  print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
  print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
  print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
  print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))
"""
   ------------- for : 0.65---------------
Accuracy : 0.43256666666666665
Macro f1 score : 0.501832360150235
Micro f1 scoore : 0.567803603355797
Hamming loss : 0.00838
 ------------- for : 0.7---------------
Accuracy : 0.4325333333333333
Macro f1 score : 0.5040089974405092
Micro f1 scoore : 0.5686298056303718
Hamming loss : 0.008381666666666667
 ------------- for : 1---------------
Accuracy : 0.4307666666666667
Macro f1 score : 0.5091321291199462
Micro f1 scoore : 0.5703628348592744
Hamming loss : 0.008446666666666667
"""

 ------------- for : 0.65---------------
Accuracy : 0.1
Macro f1 score : 0.0
Micro f1 scoore : 0.0
Hamming loss : 0.013666666666666667
 ------------- for : 0.7---------------
Accuracy : 0.1
Macro f1 score : 0.0
Micro f1 scoore : 0.0
Hamming loss : 0.013666666666666667
 ------------- for : 1---------------
Accuracy : 0.1
Macro f1 score : 0.01
Micro f1 scoore : 0.047619047619047616
Hamming loss : 0.013333333333333334


'\n   ------------- for : 0.65---------------\nAccuracy : 0.43256666666666665\nMacro f1 score : 0.501832360150235\nMicro f1 scoore : 0.567803603355797\nHamming loss : 0.00838\n ------------- for : 0.7---------------\nAccuracy : 0.4325333333333333\nMacro f1 score : 0.5040089974405092\nMicro f1 scoore : 0.5686298056303718\nHamming loss : 0.008381666666666667\n ------------- for : 1---------------\nAccuracy : 0.4307666666666667\nMacro f1 score : 0.5091321291199462\nMicro f1 scoore : 0.5703628348592744\nHamming loss : 0.008446666666666667\n'

In [ ]:
from sklearn.svm import SVC
svct=SVC(kernel='rbf')
svc = OneVsRestClassifier(svct)

svc.fit(X_train_multilabel, y_train)
y_pred=svc.predict(X_test_multilabel)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.3626666666666667


In [ ]:
from sklearn.svm import SVC
param_grid = {'estimator__C': [1, 100, 1000],
              'estimator__gamma': ['scale','auto',1,.1, .01,.001 ],
              'estimator__kernel': ['rbf']
              }
svc = OneVsRestClassifier(SVC())
rbf = model_selection.GridSearchCV(estimator=svc, param_grid=param_grid, verbose = 10)
rbf.fit(X_train_multilabel, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] estimator__C=1, estimator__gamma=scale, estimator__kernel=rbf ...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  estimator__C=1, estimator__gamma=scale, estimator__kernel=rbf, score=0.204, total=   3.5s
[CV] estimator__C=1, estimator__gamma=scale, estimator__kernel=rbf ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.5s remaining:    0.0s


[CV]  estimator__C=1, estimator__gamma=scale, estimator__kernel=rbf, score=0.185, total=   3.5s
[CV] estimator__C=1, estimator__gamma=scale, estimator__kernel=rbf ...


...

[CV]  estimator__C=1, estimator__gamma=auto, estimator__kernel=rbf, score=0.130, total=   0.3s
[CV] estimator__C=1, estimator__gamma=auto, estimator__kernel=rbf ....


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   18.8s remaining:    0.0s


[CV]  estimator__C=1, estimator__gamma=auto, estimator__kernel=rbf, score=0.148, total=   0.3s
[CV] estimator__C=1, estimator__gamma=1, estimator__kernel=rbf .......
[CV]  estimator__C=1, estimator__gamma=1, estimator__kernel=rbf, score=0.204, total=   3.3s
[CV] estimator__C=1, estimator__gamma=1, estimator__kernel=rbf .......
[CV]  estimator__C=1, estimator__gamma=1, estimator__kernel=rbf, score=0.185, total=   3.4s
[CV] estimator__C=1, estimator__gamma=1, estimator__kernel=rbf .......
[CV]  estimator__C=1, estimator__gamma=1, estimator__kernel=rbf, score=0.111, total=   3.5s
[CV] estimator__C=1, estimator__gamma=1, estimator__kernel=rbf .......
[CV]  estimator__C=1, estimator__gamma=1, estimator__kernel=rbf, score=0.130, total=   3.4s
[CV] estimator__C=1, estimator__gamma=1, estimator__kernel=rbf .......
[CV]  estimator__C=1, estimator__gamma=1, estimator__kernel=rbf, score=0.148, total=   3.3s
[CV] estimator__C=1, estimator__gamma=0.1, estimator__kernel=rbf .....
[CV]  estimator__C=

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  3.6min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=OneVsRestClassifier(estimator=SVC(C=1.0,
                                                         break_ties=False,
                                                         cache_size=200,
                                                         class_weight=None,
                                                         coef0=0.0,
                                                         decision_function_shape='ovr',
                                                         degree=3,
                                                         gamma='scale',
                                                         kernel='rbf',
                                                         max_iter=-1,
                                                         probability=False,
                                                         random_state=None,
                                                         shrinking=True,
            

In [ ]:
print(rbf.best_params_)
print(rbf.best_estimator_)
y_pred = rbf.predict(X_test_multilabel)
print("Accuracy :",metrics.accuracy_score(y_test,y_pred))
print("Macro f1 score :",metrics.f1_score(y_test, y_pred, average = 'macro'))
print("Micro f1 scoore :",metrics.f1_score(y_test, y_pred, average = 'micro'))
print("Hamming loss :",metrics.hamming_loss(y_test,y_pred))

{'estimator__C': 1, 'estimator__gamma': 'scale', 'estimator__kernel': 'rbf'}
OneVsRestClassifier(estimator=SVC(C=1, break_ties=False, cache_size=200,
                                  class_weight=None, coef0=0.0,
                                  decision_function_shape='ovr', degree=3,
                                  gamma='scale', kernel='rbf', max_iter=-1,
                                  probability=False, random_state=None,
                                  shrinking=True, tol=0.001, verbose=False),
                    n_jobs=None)
Accuracy : 0.1
Macro f1 score : 0.0
Micro f1 scoore : 0.0
Hamming loss : 0.013666666666666667
